#  Simulation Runner for LucasShark AMMPS Simulation - v0.0.1

In [1]:
from simRunner import *

SyntaxError: invalid syntax (simRunner.py, line 1337)

## Aim  
We will test that the ammps market can produce results that are in agreement with the Lucas asset pricing model. When the market makers have market maker Lucas factor = 1 they will only consider the dividend for pricing, while if it's 0 they will only consider their inventory. moving the factor away from 1 should create deviations from the Lucas pricing model. Even when the factor is 0 we will expect some degree of alignment between the prices and the Lucas pricing model as the institutional investors are using the Lucas pricing model too.  
  
While the institutional investors are Lucas asset pricers, they have some dispersion of their fair price. This is set by the inst_val_std parameter. We want to see how that dispersion affect the prices and trading.  
  
## Setup  
### AMMPS:  
As usual get the latest binaries from the master branch of ammps_sharkfin_container  
  
AMMPS comes with a number of new runtime options for outputs.  
  
The most important is the -c option, if it is set to true the logger write a compressed csv file. This saves a large amount of space and avoids us having to compress later. This comes with a small overhead on the cpu but if we are aiming for 1.1 core per simulation we should be covered.  
  
A second new option that might be useful is the -p option that adds a prefix to the logs written by ammps.  
  
For running a Lucas shark simulation with ammps only, we need to set an option to simulate a connection with SHARKfin this is done by setting the -s option to true. This option also comes with the -v and -d options that control the daily volume passed to the broker institutions simulating volume from the consumers in SHARKFin. In this case we just set both to 0.0. When using this option there should be no option to specify a RPC host etc.  
  
For running a Lucas Shark 0 simulation with ammps only we need to run:  
  
`dotnet [path_to]amm.engine.dll RunConfFromFile config_file output_directory -n unique_id_number [-c true] -s true -v 0.0 -d 0.0 [-p prefix]`  
  
I suggest using the compression option. Prefix option can be left out if you don't think it is usefull.  
  
### AMMPS Configuration Generator  
The ammps configuration generator generates a configuration file for the specific ammps simulation, this contains simulation specific parameters such as the period to be simulated, the random seed to be used, and the configurations for the agents in the simulations. It is now packed as a python package that can be installed in a python environment, or the scripts can be run using python.  
  
Scripts for generating the configs is found in the acg/simulations subdir.  
  
For the Lucas shark tests we need to run  
  
`make_lucas_shark_config.py [-h] [--out-dir CONF_DIR] --name RUN_NAME --seed SEED [--days NUMBER_OF_DAYS] --mm_lucas_factor MM_LUCAS_FACTOR [--mm_size MM_SIZE] [--inst_val_std DIVIDEND_VAL_STD]`  
  
The new options are --mm_lucas_factor MM_LUCAS_FACTOR [--mm_size MM_SIZE] [--inst_val_std DIVIDEND_VAL_STD]  
  
### Simulation Grid  
We want to test a simulation grid where we change mm_lucas_factor and inst_val_std, and seed.  
  
I propose we use 10 seeds, and mm_lucas_factor =[0.0,0.25,0.5,0.75,1.0] and inst_val_std = [0.01, 0.05, 0.1, 0.15, 0.2]  
  
This will require 5x5X10 = 250 simulations, of about 30 minutes each.  


## Configure Parameters

In [6]:
#experiment parameters
experimentName="lucasShark"
tag ='ammps_only'
quarters=4
#Replicant Parameters:
seedcount=10
seedKey=6174
mm_lucas_factor =[0.0,0.25,0.5,0.75,1.0] 
inst_val_std = [0.01, 0.05, 0.1, 0.15, 0.2]
quarters=4



## Build Simulation Objects

In [7]:
#create entities in python list

simEntities = build_ammps_only_configs(experimentName,seedcount,seedKey,market_fraction_values,ammps_volume_values,quarters,tag)

len(simEntities)


Generating simulations using the following seeds:[ 4343 14751 45561 23932 88034 66505 94252 20490 55174 12660 82502 66772
 56957 20297 32128 19680 53789  7432 50806   242 90174 76009 57420 63058
 88332 56152 50906 67767  6626 63295 34915 68355 60668 35338 58882 51316
 38514 38782 34015 37760 12741 48582 79115 66460 29346 31288 29668 24793
 62062 42532]


750

In [8]:
simEntities[5]

{'PartitionKey': '6',
 'RowKey': 'seedggammps6|66505|Market_Fraction|0ammps_volume|5000',
 'experimentName': 'seedggammps',
 'status': 'pending',
 'simid': 6,
 'market_fraction_value': 0,
 'quarters': 4,
 'tag': 'ammps_updates',
 'ammps_config_cmd': '/usr/bin/python /usr/simulation/ammps_config_generator/make_ggshark_config.py --seed 66505 --name test_conf6.xlsx --days 240 --market_fraction 0 --out-dir /usr/simulation/ ',
 'start_ammps_cmd': 'dotnet /usr/simulation/ammps_bin/amm.engine.dll RunConfFromFile /usr/simulation/test_conf6.xlsx /mnt/sharkfin-ammps-fs/ammps_data_seedggammps6out --instruments ABC:NYSE:0.01:0.01,ABC:CITADEL:0.001:0.001 --number 6 --simulated-rpc true --simulated-rpc-buy-sell-vol 5000 --simulated-rpc-buy-sell-vol-std 1000 ',
 'start_sharkfin_cmd': '/usr/bin/python /usr/simulation/SHARKFin/simulate/run_any_simulation.py /shared/home/ammpssharkfin/output/seedggammps6 --simulation Attention --tag ammps_updates --seed 66505 --quarters 4 ',
 'seed': '66505',
 'sharkfin

## Load Simulation Objects in Azure Tables

In [9]:
experimentName
!az login   
#!az login --use-device-code`

A web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
The following tenants require Multi-Factor Authentication (MFA). Use 'az login --tenant TENANT_ID' to explicitly login to a tenant.
72f988bf-86f1-41af-91ab-2d7cd011db47 'Microsoft'
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "7cf48d45-3ddb-4389-a9c1-c115526eb52e",
    "id": "0abf6ad3-c04d-4533-8d9b-9d9c3e79d822",
    "isDefault": false,
    "managedByTenants": [],
    "name": "UP-OIT-Nittany-AI-Advance",
    "state": "Enabled",
    "tenantId": "7cf48d45-3ddb-4389-a9c1-c115526eb52e",
    "user": {
      "name": "wjt5121@psu.edu",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "7cf48d45-3ddb-4389-a9c1-c115526eb52e",
    "id": "39bf3a99-0a18-494c-ac4b-dcabb7f9e285",
 

In [10]:
#!az login
vaultName = 'sharkfinkv'
#names of needed secerts and keys - These can be located/updated here  ----> sharkfinkv.vault.azure.net
connectionName = 'simulationdataConnectionString'
#create table
newTable = create_table(experimentName,vaultName,connectionName)
#create a simulation grid clss object to store the simulation configurations
simTable = simulationGrid(experimentName, vaultName, connectionName)


opened vault, retrived secrect string
opened vault, retrived secrect string


In [11]:
#create entities in Azure Table (csv list)
for sim in simEntities:
    create_table_entity(simTable,sim)

{'date': datetime.datetime(2023, 3, 31, 12, 48, 16, tzinfo=datetime.timezone.utc), 'etag': 'W/"datetime\'2023-03-31T12%3A48%3A16.9501467Z\'"', 'version': '2019-02-02'}
{'date': datetime.datetime(2023, 3, 31, 12, 48, 16, tzinfo=datetime.timezone.utc), 'etag': 'W/"datetime\'2023-03-31T12%3A48%3A16.9941219Z\'"', 'version': '2019-02-02'}
{'date': datetime.datetime(2023, 3, 31, 12, 48, 16, tzinfo=datetime.timezone.utc), 'etag': 'W/"datetime\'2023-03-31T12%3A48%3A17.036099Z\'"', 'version': '2019-02-02'}
{'date': datetime.datetime(2023, 3, 31, 12, 48, 16, tzinfo=datetime.timezone.utc), 'etag': 'W/"datetime\'2023-03-31T12%3A48%3A17.0780751Z\'"', 'version': '2019-02-02'}
{'date': datetime.datetime(2023, 3, 31, 12, 48, 16, tzinfo=datetime.timezone.utc), 'etag': 'W/"datetime\'2023-03-31T12%3A48%3A17.1170541Z\'"', 'version': '2019-02-02'}
{'date': datetime.datetime(2023, 3, 31, 12, 48, 16, tzinfo=datetime.timezone.utc), 'etag': 'W/"datetime\'2023-03-31T12%3A48%3A17.1580311Z\'"', 'version': '2019-0

### View the simulations as a Datafram using the following cell.

In [ ]:

#retrives entities from Table
simgrid = simulationGrid(experimentName,vaultName,connectionName)
entities = simgrid.table_client.list_entities()
tableEntities = list(entities)
print(f"Located {len(tableEntities)} entities.")

edf = pd.DataFrame(tableEntities)
print(edf.info())
print(edf.head())

for key in tableEntities[1]:
    print(f"{key}={tableEntities[2][key]}")

In [ ]:
edf = pd.DataFrame(simEntities)
print(edf.info())
print(edf.head())
#display item details from Azure table
edf.loc[2]['start_ammps_cmd']
#edf.loc[edf['simid'] == 2]


## Connect/Verify Slurm Cluster size/availabilty

* **CycleServer: https://13.92.241.145/home **
    * [Username/Passord: ammpssharkfin ammpsshrk1qaz@WSX
  PEM key is simEngine.pem  
Update the address and PEM key for your scheduler and in the next cell and test communication.


In [16]:
#from simRunner import *
#udate the variables to match the slurm environment
#ssh -i <private key path> azureuser@cycleshark.eastus.cloudapp.azure.com
arctic = '13.90.145.165'
ggammps ='40.88.138.86'
ammplantic= '20.25.24.15'
cyclecloud='13.92.241.145'
southern = '20.172.215.89'
user = 'ammpssharkfin'
#download pem keys from Azure Key vault. Only needs to occur if the keys are not on the local machine
#cyclevmkey = get_azKey(vaultName, cloudCycleVMk)

#!chmod 400 cloudcycle-vm_key.pem
#!chmod 400 cycleShark_key.pem
#!chmod 400 slurms-RabbitMQ_key.pem
#!chmod 400 simEngine.pem
#!ls -l {cyckey}
#clusterkey = get_azKey(vaultName, clusterVMs)
#rabbitkey = get_azKey(vaultName, rabbitMQKey)

#location of pem keys
#cyckey = 'cloudcycle-vm_key.pem'
#clustkey = 'cycleShark_key.pem'
#rabkey = 'slurms-RabbitMQ_key.pem'
simengine = 'simEngine.pem'
#valid ssh keys are present. Un comment Download from keyvault if not present uncomment lines 9-11 and reun cell again.

#!ls -l {cyckey}
#!ls -l {clustkey}
#!ls -l {rabkey}

    

## Push node configuration and Requirements files to the GGAMMPS scheduler
###Nodes are configured with a start-up script which is stored on the CucleCloud server in this location. -> /opt/cycle_server/work/staging/projects/slurm/2.6.2/default/cluster-init/scripts) 

The requirments.txt file is located here -> /opt/cycle_server/work/staging/projects/slurm/2.6.2/default/cluster-init/files/requirments.txt

In [17]:

sbatchLocalPath = f'{experimentName}job.sh'
sbatchRemotePath = f'/shared/home/ammpssharkfin/{experimentName}job.sh'
runnerLocalPath = 'slurmRunnerAMMPS.py'
runnerRemotePath = '/shared/home/ammpssharkfin/slurmRunner.py'
clusterInitLocalPath = ''
clusterInitPath = '/opt/cycle_server/work/staging/projects/slurm/2.6.2/default/cluster-init/scripts/'
requirementsLocalPath = 'requirements.txt'
requirmentsPath = '/shared/home/ammpssharkfin/'
configFile = '01-os-config-alma-linux.sh'
simengine = 'simEngine.pem'




In [18]:
#Runs the getScheduleStatus function to validate connectivity to the the scheduler.
getSchedulerStatus(southern, simengine, user)

Attempting to connect to 20.172.215.89.......
Connected
There are 0 items in the queue.
ssh -i simEngine.pem ammpssharkfin@20.172.215.89 sinfo
       1

Scheduler 20.172.215.89 is ready.


In [19]:
#create initial directory structure on new scheduler
cmd = 'mkdir output;mkdir output/logs;mkdir output/logs/slurm; mkdir output/logs/sharkfin; mkdir output/logs/ammps_conf; mkdir output/logs/ammps'
#cmd = 'ls -R output'
run_cmd_remote(southern, user, simengine, cmd)

mkdir output;mkdir output/logs;mkdir output/logs/slurm; mkdir output/logs/sharkfin; mkdir output/logs/ammps_conf; mkdir output/logs/ammps


('', '')

In [ ]:
#copy approriate slurmRunner.py for experiment. sharkFin/ammps vs ammps vs sharkFin
sendslurm = submit_file_to_remote(southern, user, simengine, runnerLocalPath, runnerRemotePath, execute=False)

In [ ]:
#copy requirements.txt file to share 
sendslurm = submit_file_to_remote(ammpslantic, user, simengine, requirementsLocalPath, requirmentsPath, execute=False)

## Generate sbatch script required to submit Jobs to GGAMMPS Slurm Scheduler

In [20]:
#this function will create a sbatch script job,simcount is the number of simulation in the array you would like to run. 
simcount = 750
job_cmd = generate_slurm_job(experimentName, simcount)
job_cmd

'#!/bin/bash\n#SBATCH --job-name=seedggammps_Slurm_Job\n#SBATCH --nodes=1\n#SBATCH --ntasks-per-node=1\n#SBATCH --cpus-per-task=1\n#SBATCH --array=1-750\n#SBATCH --time=24:00:00\n#SBATCH --output=./output/logs/slurm/seedggammpsjob_%A_%a.out.log\n#SBATCH --error=./output/logs/slurm/seedggammpsjob_%A_%a.err.log\necho "Strart of bash script. Setting environmental variables."\ndate -u\nSIMID=$SLURM_ARRAY_TASK_ID\nEXPERIMENTNAME=seedggammps\necho "Staring simulation $SIMID plese wait."\ndate -u\npython3 /shared/home/ammpssharkfin/slurmRunner.py $EXPERIMENTNAME $SIMID\nwait\necho "slurmRunner.py has returned! Bash script exiting."\ndate -u'

In [ ]:
sbatchLocalPath

In [21]:
#create sbatch script used to execute simulation on slurm clustrer. if execute option is True jobs will be submited. 
out, err = submit_file_to_remote(southern, user, simengine, sbatchLocalPath, sbatchRemotePath, execute=True)

In [27]:
if out:print(out)
if err:print(err)

Submitted batch job 1



In [ ]:
#todo move to container

### Setup info for cyclecloud and scheduler connections and paths.

#### Once connection to the Scheduler node has been established you are able to submit Experiments to be exicuted on the slurm cluster nodes

## Submit simulations jobs to Slurm Scheduler

#todo Scheduler init needs to have creation of output and logs directories.

-Azure CycleCloud Server - Provides the Compute in the form of Slurm Clusters
-Slurm HPC Clusters
-Azure Storage Account - Blob and Files

-Customization script to config cluster nodes (located here -> /opt/cycle_server/work/staging/projects/slurm/2.6.2/default/cluster-init/scripts)
    -Linux VM w/ Python3.9 + requirements.txt 
    -requirments.txt is located here -> /opt/cycle_server/work/staging/projects/slurm/2.6.2/default/cluster-init/files/requirments.txt
    -Cloning of the follow git repos
        https://github.com/mesalas/ammps_sharkfin_container.git /usr/simulation/ammps_bin    
        https://github.com/mesalas/ammps_config_generator /usr/simulation/ammps_config_generator
        https://github.com/sbenthall/SHARKFin /usr/simulation/SHARKFin/
        https://github.com/econ-ark/HARK.git /usr/simulation/harkrepo
        Note we must recopy the hark files - (investigate checking out the branch'sudo -H cp -Rf /usr/simulation/harkrepo/HARK/ /shared/home/ammpssharkfin/.local/lib/python3.9/site-packages/



In [ ]:
#todo copy slurmrunner7 to the schedulers, copy it to the notebok server, 
#todo have the requirments.txt file be staged on scheduler node nfs share
#todo copy the cluster configuration file to the notebook server
#todo updte simrunner to update azure table with data location.


### Use the generate_slurm_job function to configure the slurm job script used issue the simulation jobs to the scheduler.

In [ ]:

job_cmd = generate_slurm_job(experimentName, 750)
job_cmd

In [ ]:
out = submit_file_to_remote(arctic, user, simengine, sbatchlocalpath, sbatchremotepath, execute=False)

#### Copy slurmRunner script to share

In [ ]:
arctic

### Use the submit_job function to copy the job script to the scheduler and then execute it. 

In [ ]:
out, err = submit_file_to_remote(arctic, user, simengine, sbatchlocalpath, sbatchremotepath, execute=True)

In [ ]:
if out:print(out)
getSchedulerStatus(arctic, sshkeypath, user)
if err:print(err)

In [ ]:
if out:print(out)
if err:print(err)

## Monitor simulation progress

In [ ]:
def getSlurmJobLogs(schedulerIP,experimentName, jobID, simID, clusterKey, user):
    logDir = '/output/logs/slurm/'
    slurmOut = f"{logDir}{experimentName}job_{jobID}_{simID}.out.log"
    slurmErr = f"{logDir}{experimentName}job_{jobID}_{simID}.out.log"
    
    scmd= f"cat {slurmOut}"
    runner = f"ssh -o StrictHostKeyChecking=no -i {clusterKey} {user}@{schedulerIP} {scmd}"
    out, err = subprocess.Popen(scmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True).communicate()
    if out:
        print(f"Attempting to connect to {schedulerIP}.......")
        print("Connected")
        print(out)
    if err:
        print(err)


In [ ]:
atlanticScheduler = '52.255.135.7'
pacificScheduler = '172.174.84.253'
arctic = '137.135.86.1'
cyckey = 'cloudcycle-vm_key.pem'
clustkey = 'cycleShark_key.pem'
rabkey = 'slurms-RabbitMQ_key.pem'
localpath = f'{experimentName}job.sh'
sbatchremotepath = '/shared/home/ammpssharkfin/'
user = 'ammpssharkfin'
key=clustkey

#/shared/home/ammpssharkfin/output/logs/slurm/slurm39ex10job_82_9.out.log
                   #        /output/logs/slurm/slurm39ex10job_82_9.out.log
#slurm39ex10job_2_9.out.log

In [ ]:
jobID = 82
simID = 9
slog10 = getSlurmJobLogs(arctic,'slurm39ex10', jobID, simID, clustkey, user)
#slog11 = getSlurmJobLogs(arctic,experimentName, jobID, simID, clustkey, user)
slog10

In [ ]:
/output/logs/slurm/{experimentName}job_{jobID}_{simID}.out.log
#SBATCH --error=./output/logs/slurm/{experiment_name}job_%A_%a.err.log

In [ ]:
out, err = subprocess.Popen(scmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,encoding="utf-8").communicate()






In [ ]:
if out:
    print(out)
if err:
    print(err)


## Process simulation output

In [ ]:
#cmd =  'ls output/simulation/'
cmd =  './output/logs/slurm'
scmd = f"ssh -o StrictHostKeyChecking=no -i {clustkey} {user}@{atlanticScheduler} {cmd}"
out, err = subprocess.Popen(scmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True).communicate()

# list all files
fs = out.split('\n')

print(fs)

#for f in fs:
    
     # extract gzips if there are any
#    if f.startswith("whiteShark") and f.endswith(".gz"):
#        gzipF = "/shared/home/ammpssharkfin/output/simulation/" + f
#        cmd =  'gzip -d ' + gzipF

#        scmd = f"ssh -o StrictHostKeyChecking=no -i {clustkey} {user}@{atlenticScheduler} {cmd}"
#        out, err = subprocess.Popen(scmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True).communicate()
#        print("unzip", f)

    # update this to not hard code whiteShark
#    if f.startswith("whiteShark") and not f.endswith(".gz") and "sim_stats" in f:
        
#        cmd =  'cat ' + "output/simulation/" + f
#        scmd = f"ssh -o StrictHostKeyChecking=no -i {clustkey} {user}@{atlenticScheduler} {cmd}"
#        out, err = subprocess.Popen(scmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True).communicate()

#        cmd_class =  'cat ' + "output/simulation/" + f[:-14] + "_class_stats.csv"
#        scmd_class = f"ssh -o StrictHostKeyChecking=no -i {clustkey} {user}@{atlenticScheduler} {cmd_class}"
#        out_class, err = subprocess.Popen(scmd_class, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True).communicate()      

#        process_run(out, f, out_class)
        


## Visualize Results

In [ ]:
from simRunner import *
rdata = pd.read_csv('all_clean.txt')
rdata = rdata.drop(columns="Unnamed: 0")

rdata["att_cat"] = rdata['attention'].apply(lambda x: str(x) + ' ')
rdata["mock"] = rdata['seconds'] < 100

data = rdata[~rdata["mock"]]
 
sns.displot(
    rdata, 
    x="ror_volatility", 
    hue="attention", 
    element='step'
)

In [ ]:
rdata